In [96]:
import sys
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import Stream
import string

consumer_key = "aT1NWOoMyAJ03Kkk98hAkfIbV"
consumer_secret = "YIqXnd6ggBLMxydU7avzaNLNOepH1k7FFuMIwg1YgLowYBm7LT"
access_token = "76361411-ReKKExqAKmlmIl3PZn3X11RCHwBBji4ORkUahe700"
access_token_secret = "Rk5XIyMptqJCD9Gfe2JJa3RZDdHhzecuNBbjIEE0TCm2R"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth, wait_on_rate_limit=True)

In [97]:
# stream Turkish tweets, save the userids of users that post tweets in turkish_users

turkish_users = []
num_user = 0

class Listener(tweepy.Stream):
    def on_status(self, status):
        global num_user 
        userid = str(status.user.id_str)
        tweet_text = status.text
        turkish_users.append(userid)
        num_user += 1
        print(num_user)
        #print("--" + tweet_text + "\n")
        #print(userid + "--" + tweet_text + "\n")

    def on_error(self, status_code):
        print(status_code)
        return False
    
stream = Listener(
  consumer_key, consumer_secret,
  access_token, access_token_secret
)

try:
    print('Start streaming.')
    stream.sample(languages=['tr'])
except KeyboardInterrupt:
    print("Stopped.")
    print(turkish_users)

Start streaming.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
Stopped.
['1253504616337158144', '1461654118309711877', '1440692213772533764', '1351220563142987778', '1464643436145950723', '575049291', '1031173381', '1235593354504781832', '1135266924139548672', '1449234204944969728', '1460591305315303430', '1460409984618053644', '1437491348412772352', '1444530055380672513', '867485259046281216', '1460435907425751055', '1460452387508862979', '1460670722896969728', '1380141681270489088', '1440984880536604675', '1460490145656889348', '1444302586438471684', '2703016635', '1432045239062175747', '1461654000529559559', '1461371227868643337', '1461378272915333123', '1397887738943262720', '968616131870289925', '1460483582871408641', '1276646563503394817', '1460483913365831680', '1399188979', '1439674856983957507', '1460486115098513414', '1460490757417091072', '1460029455489974274', '1182299167688921

In [98]:
# function that returns information about 10 latest tweets of a user, all in one row of csv
# tweet data dictionary (status object): https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet

sep = ','
def get_tweets(userid):
        sleep_on_rate_limit = False

        # 10 tweets to be extracted
        number_of_tweets=10

        tweets = api.user_timeline(user_id=userid, count=number_of_tweets, tweet_mode='extended')
        
        # Empty Array
        row = [] 
        
        #print("printing 10 tweets from useid: " + userid)
        
        for tweet in tweets:
            tweetid = str(tweet.id_str)
            userid = str(tweet.user.id)
            user_display_name = str(tweet.user.name).translate(str.maketrans('', '', string.punctuation));
            user_screen_name = str(tweet.user.screen_name).translate(str.maketrans('', '', string.punctuation));
            user_reported_location = str(tweet.user.location).translate(str.maketrans('', '', string.punctuation));
            user_profile_description = str(tweet.user.description).translate(str.maketrans('', '', string.punctuation));
            user_profile_url = "remove" 
            follower_count = str(tweet.user.followers_count)
            following_count = str(tweet.user.friends_count)
            account_creation_date = str(tweet.user.created_at)
            account_language = str(tweet.user.lang)
            tweet_language = str(tweet.lang)
            tweet_text = str(tweet.full_text).translate(str.maketrans('', '', string.punctuation));
            tweet_time = str(tweet.created_at)
            tweet_client_name =  str(tweet.source)
            in_reply_to_userid = "remove" # str(tweet.in_reply_to_user_id)
            in_reply_to_tweetid ="remove" # str(tweet.in_reply_to_status_id)
            quoted_tweet_tweetid = "remove"
            
            if tweet.retweet_count > 0:
                is_retweet = "true" # or tweet.retweeted_status or str(tweet.retweeted)
            else:
                is_retweet = "false"
            #is_retweet = str(tweet.retweeted_status)
            retweet_userid = "remove"
            retweet_tweetid = "remove"
            latitude = "remove" #str(tweet.coordinates)
            longitude = "remove" #str(tweet.coordinates)
            quote_count = "remove" #str(tweet.quote_count)
            reply_count = "remove" #str(tweet.reply_count)
            like_count = str(tweet.favorite_count)
            retweet_count = str(tweet.retweet_count)
                        
            hashinfo = tweet.entities['hashtags']
            urlinfo = tweet.entities['urls']
            usermentioninfo = tweet.entities['user_mentions']
            
            hashtags_list = list()
            for j in range(0, len(hashinfo)):
                hashtags_list.append(hashinfo[j]['text'])
            hashtags = ",".join(hashtags_list)
            
            urls_list = list()
            for j in range(0, len(urlinfo)):
                urls_list.append(urlinfo[j]['url'])
            urls = ",".join(urls_list)
            
            usermentions_list = list()
            for j in range(0, len(usermentioninfo)):
                usermentions_list.append(usermentioninfo[j]['id_str'])
            user_mentions = ",".join(usermentions_list)
            
            row = ('"' + tweetid + '"' + sep + '"' + userid + '"' + sep + '"' + 
            user_display_name + '"' + sep + '"' + user_screen_name + '"' + sep + '"' + user_reported_location + 
            '"' + sep + '"' + user_profile_description + '"' + sep + '"' + user_profile_url + '"' + 
            sep + '"' + follower_count + '"' + sep + '"' + following_count + '"' + sep + '"' + account_creation_date + 
            '"' + sep + '"' + account_language + '"' + sep + '"' + tweet_language + '"' + sep + '"' + tweet_text + '"' + 
            sep + '"' + tweet_time + '"' + sep + '"' + tweet_client_name + '"' + sep + '"' + in_reply_to_userid + '"' + sep + 
            '"' + in_reply_to_tweetid + '"' + sep + '"' + quoted_tweet_tweetid + '"' + sep + '"' + is_retweet + '"' + sep + 
            '"' + retweet_userid + '"' + sep + '"' + retweet_tweetid + '"' + sep + '"' + latitude + '"' + sep + '"' + 
            longitude + '"' + sep + '"' + quote_count + '"' + sep + '"' + reply_count + '"' + sep + '"' + like_count + 
            '"' + sep + '"' + retweet_count + '"' + sep + '"' + hashtags + '"' + sep + '"' + urls + '"' + 
            sep + '"' + user_mentions + '"'"\n")
    
            csv.write(row)
    
            #print(row)

In [99]:
print(len(turkish_users))

51


In [100]:
# for each user in turkish_users, save their last 10 tweets in a csv file by calling get_tweets('userid')

csv = open('test2.csv','a')

sleep_on_rate_limit = False

header = ('"tweetid"' + sep + '"userid"' + sep + '"user_display_name"' + sep +
            '"user_screen_name"' + sep + '"user_reported_location"' + sep + '"user_profile_description"' + sep +
          '"user_profile_url"' + sep + '"follower_count"' + sep + '"following_count"' + sep + 
          '"account_creation_date"' + sep + '"account_language"' + sep + '"tweet_language"' + sep + 
          '"tweet_text"' + sep + '"tweet_time"' + sep + '"tweet_client_name"' + sep + '"in_reply_to_userid"' + 
          sep + '"in_reply_to_tweetid"' + sep + '"quoted_tweet_tweetid"' + sep + '"is_retweet"' + sep + 
          '"retweet_userid"' + sep + '"retweet_tweetid"' + sep + '"latitude"' + sep + '"longitude"' + sep + 
          '"quote_count"' + sep + '"reply_count"' + sep + '"like_count"' + sep +'"retweet_count"' + sep + 
          '"hashtags"' + sep + '"urls"' + sep + '"user_mentions"\n')
          

csv.write(header)

for userid in turkish_users:
    get_tweets(userid)
